# 30강 메일 발송 자동화

- SMTP 서버
  - 외부 프로그램의 요청에 따라 메일을 발송하고 응답을 반환하는 시스템
  - 파이썬 자체는 메일을 발송할수 없어 외부 SMTP와 연동을 통해 메일을 발송

In [1]:
# -> 경로 정보를 취득하기 위한 모듈
import os.path
# -> 발송서버와 연동하기 위한 모듈
from smtplib import SMTP
# -> 본문 구성 기능
from email.mime.text import MIMEText
# -> 파일을 Multipart 형식으로 변환
from email.mime.application import MIMEApplication
# -> 파일을 본문에 추가하는 기능 제공
from email.mime.multipart import MIMEMultipart

In [2]:
# 보내는 사람 주소 (로그인 계정과 동일해야 함)
from_addr = "yijingue@naver.com"

# 받는 사람 주소
to_addr = "yijingue@gmail.com"

# 제목
subject = "파이썬 메일 발송 테스트 입니다."

# 본문내용 작성 -> 삼중 따옴표 안에서는 줄바꿈이 자유롭다
content = """안녕하세요. 파이썬으로 발송하는 메일입니다.

잘 발송되는지 확인해보도록 하겠습니다.

하나 둘 ~ 셋"""

# 첨부파일이 필요 없는 경우
# files = []

# 첨부파일이 필요한 경우 (파일이 실제로 존재해야 함)
files = ["hello.txt", 'world.txt']

In [3]:
# 컨텐츠 형식 (plain or html)
content_type = "plain"

# 로그인 계정 이름(네이버 = 아이디, 구글 = 메일주소)
username = ""

# 비밀번호 (네이버 = 개인비밀번호, 애플리케이션 비밀번호, 구글 = 앱 비밀번호)
password = ""

# 구글 발송 서버 주소와 포트 (고정값)
# smtp = "smtp.gmail.com"
# port = 587

# 네이버 발송 서버 주소와 포트 (고정값)
smtp = "smtp.naver.com"
port = 465

In [4]:
# 메일 발송 정보를 저장하기 위한 객체
msg = MIMEMultipart()

msg['Subject'] = subject # 메일 제목
msg['From'] = from_addr # 보내는 사람
msg['To'] = to_addr # 받는 사람

# 본문 설정 -> 메일의 내용과 형식 지정
msg.attach(MIMEText(content, content_type))

In [5]:
# 리스트 변수의 원소가 하나라도 존재할 경우 True
if files:
    for f in files:
        # 바이너리(b) 형식으로 읽기(r) : 바이너리 읽기는 인코딩 설정 필요X
        with open(f, 'rb') as a_file:
            # 전체 경로에서 파일의 이름만 추출
            basename = os.path.basename(f)
            # 파일의 내용과 파일이름을 메일에 첨부할 형식으로 변환
            part = MIMEApplication(a_file.read(), Name = basename)

            # 파일첨부
            part['Content-Disposition'] = 'attachment; filename="%s"' % basename
            msg.attach(part)

In [6]:
mail = SMTP(smtp)
# 메일 서버 접속
mail.ehlo()
# 메일 서버 연동 설정
mail.starttls()
# 메일 서버 로그인
mail.login(username, password)
# 메일 보내기
mail.sendmail(from_addr, to_addr, msg.as_string())
# 메일 서버 접속 종료
mail.quit()

(221, b'2.0.0 Closing connection CEub39fKSuC-byOhqb7IrA - nsmtp')

# 30-1 메일링 리스트

In [1]:
import MyMailer
import datetime as dt

In [2]:
today = dt.datetime.now()
year = today.year
month = today.month
day = today.day
print(year, month, day)

2023 11 30


In [6]:
# 메일 발신자 정보
fromAddr = "yijingue@naver.com"

# 메일 제목을 위한 템플릿
subjectTmpl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다."

In [4]:
with open('mailtest/content.txt', 'r', encoding = 'utf-8') as f:
    contentTmpl = f.read()
    print(contentTmpl)

안녕하세요 {name}님

{yy}년도 {mm}월 급여명세서와 결산보고서 보내드립니다.

귀하의 노고에 감사드립니다.

- {yy}년 {mm}월 {dd}일 / 운영지원팀 드림


In [9]:
with open("mailtest/mail_list.csv", "r", encoding = 'euc-kr') as f:
    csv = f.readlines()

    # CSV의 라인 수 만큼 반복
    for line in csv:
        # 한 줄의 내용을 분리
        name, email, file1, file2 = line.strip().split(",")
        # print(name, email, file1, file2)

        toAddr = "{email}".format(email = email)
        # print(toAddr)

        # 메일 제목
        subject = subjectTmpl.format(name = name, yy = year, mm = month)
        # print(subject)

        # 메일 내용
        content = contentTmpl.format(name = name, yy = year, mm = month, dd = day)
        # print(content)

        # 메일 보내기
        MyMailer.sendMail(fromAddr, toAddr, subject, content, [file1, file2])